In [9]:
import ee
import geemap
import folium
import time

ee.Initialize()
Map = geemap.Map()
print("Start")
# Define the region of interest of Amazon basin
# amazon_region_whole = ee.Geometry.Polygon([[[-80.0, 10.0],
#             [-44.0, 10.0],
#             [-44.0, -18.0],                           
#             [-80.0, -18.0],
#             [-80.0, 10.0]]]) 
amazon_region = ee.Geometry.Polygon([[[-80.0, 5.0],
            [-55.0, 5.0],
            [-55.0, -10.0],                           
            [-80.0, -10.0],
            [-80.0, 5.0]]]) 


# Define the bitmasks
cloud_bit_mask = ee.Number(1 << 5)  # Cloud bit is in the 6th bit position
cirrus_bit_mask = ee.Number(1 << 9)  # Cirrus bit is in the 10th bit position

# Define a masking function
def mask_edges(image):
    edge = image.lt(-30.0)  # Define an edge mask where values are less than -30
    masked_image = image.mask().And(edge.Not())  # Mask out edges
    return image.updateMask(masked_image)  # Apply the mask

# Apply the mask using bitwise AND to check that both cloud and cirrus bits are 0
def mask_clouds(image):
    qa = image.select('QA60')  # Select the QA60 band that holds cloud and cirrus bit information
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask)

# Calculate EVI
def calculate_evi(image):
    return image.expression(
        '2.5 * ((B8 - B4) / (B8 + 6 * B4 - 7.5 * B2 + 1))',
        {
            'B8': image.select('B8'),
            'B4': image.select('B4'),
            'B2': image.select('B2')
        }).rename('EVI')
    
# Function to create a grid of tiles over the region
def create_tile_grid(region, tile_size):
    bounds = region.bounds().coordinates().get(0).getInfo()
    lon_min, lat_min = bounds[0]
    lon_max, lat_max = bounds[2]

    tiles = []
    lon = lon_min
    while lon < lon_max:
        lat = lat_min
        while lat < lat_max:
            tile = ee.Geometry.Rectangle([lon, lat, lon + tile_size, lat + tile_size])
            tiles.append(ee.Feature(tile))
            lat += tile_size
        lon += tile_size

    return ee.FeatureCollection(tiles)



# Tile size in degrees (approx. 300 km × 300 km, depending on latitude)
tile_size_degrees = 2.7  # Approximate conversion for 300 km
tiles = create_tile_grid(amazon_region, tile_size_degrees)

# Apply batch processing to all tiles
# agb_maps = tiles.map(lambda tile: calculate_agb(tile))

# Process each tile and print the result
tile_list = tiles.toList(tiles.size()).getInfo()  # Convert FeatureCollection to a list
print("Size of tiles", len(tile_list))

print("Generating Map")
# Visualize the tiles on the map
Map.centerObject(amazon_region, 5);
Map.addLayer(tiles, {'color': 'red'}, 'Tile Grid');
Map


Start
Size of tiles 60
Generating Map


Map(center=[-2.5440531086315525, -62.50000000000001], controls=(WidgetControl(options=['position', 'transparen…

In [5]:
# Function to calculate AGB for each tile
def calculate_agb(tile):
    tile_geometry = ee.Feature(tile).geometry()

    # Load GEDI Level 4A data
    gedi_all = ee.FeatureCollection('LARSE/GEDI/GEDI04_A_002_INDEX')\
            .filter('time_start > "2023-01-01" && time_end < "2023-12-31"')\
            .filterBounds(tile_geometry);

    # Get the list of table_id values
    table_ids = gedi_all.aggregate_array('table_id').getInfo()

    # Initialize an empty FeatureCollection
    merged_collection = ee.FeatureCollection([])

    # Loop through each table ID and merge them
    for table_id in table_ids:
        # Load each table and merge
        table = ee.FeatureCollection(table_id).filterBounds(tile_geometry);
        gedi = merged_collection.merge(table)

    
    # print('Number of filtered GEDI points:', gedi.size().getInfo())
    print('Number of GEDI points:', gedi.size().getInfo())
    # Filter to keep only points with non-null 'agbd' values
    gedi = gedi.filter(ee.Filter.notNull(['agbd']))
    print('Number of filtered GEDI points:', gedi.size().getInfo())
    # print(gedi.first().getInfo())
    if gedi.size().getInfo() != 0:
        # Map.setCenter(-60, 5, 5);
        # Map.addLayer(merged_granules);
        # print(gedi.first().getInfo())
        # load sentinel-1 data 
        spring = ee.Filter.date('2023-03-01', '2023-04-20');
        lateSpring = ee.Filter.date('2023-04-21', '2023-06-10');
        summer = ee.Filter.date('2023-06-11', '2023-08-31');
    
        sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
                    .filterBounds(tile_geometry)\
                    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
                    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
                    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
                    .filter(ee.Filter.inList('orbitProperties_pass', ['ASCENDING', 'DESCENDING']))
        # Select the VV and VH bands
        sentinel1_vv = sentinel1.select('VV')
        sentinel1_vh = sentinel1.select('VH')
        # Apply the masking function to each image in the collection
        sentinel1_vv_masked = sentinel1_vv.map(mask_edges)
        sentinel1_vv_final = ee.Image.cat(
                sentinel1_vv_masked.filter(spring).mean(),
                sentinel1_vv_masked.filter(lateSpring).mean(),
                sentinel1_vv_masked.filter(summer).mean());
        # Apply the masking function to each image in the collection
        sentinel1_vh_masked = sentinel1_vh.map(mask_edges)
        sentinel1_vh_final = ee.Image.cat(
                sentinel1_vh_masked.filter(spring).mean(),
                sentinel1_vh_masked.filter(lateSpring).mean(),
                sentinel1_vh_masked.filter(summer).mean());
        print('Number of sentinel1 points:', sentinel1_vh_masked.size().getInfo(), sentinel1_vv_masked.size().getInfo()) 
        
        
        # Load Sentinel-2 surface reflectance data
        # Sentinel-2 Level 2A multispectral imagery, ref:  https://www.sciencedirect.com/science/article/pii/S1569843222002965#s0010
        sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                        .filterBounds(tile_geometry) \
                        .filterDate('2023-01-01', '2023-12-31') \
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))
        # Define the bitmasks
        cloud_bit_mask = ee.Number(1 << 5)  # Cloud bit is in the 6th bit position
        cirrus_bit_mask = ee.Number(1 << 9)  # Cirrus bit is in the 10th bit position
    
        sentinel2 = sentinel2.map(mask_clouds)
        # Calculate NDVI
        ndvi = sentinel2.map(lambda image: image.normalizedDifference(['B8', 'B4']).rename('NDVI')).median()
        print('Number of sentinel2 points:', sentinel2.size().getInfo()) 
       
        evi = sentinel2.map(calculate_evi).median()
        # print('Number of evi points:', evi.getInfo())
        
        
        # Load Landsat 8 Surface Reflectance data
        landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                      .filterBounds(tile_geometry) \
                      .filterDate('2023-01-01', '2023-12-31') \
                      .filter(ee.Filter.lt('CLOUD_COVER', 35))

        # Calculate NDVI for Landsat 8
        landsat_ndvi = landsat8.map(lambda image: image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')).median()

        
        
        # Load the GLO-30 DEM data from the COPERNICUS collection
        dem = ee.ImageCollection('COPERNICUS/DEM/GLO30') \
                  .filterBounds(tile_geometry) \
                  .mosaic()
        
        # Calculate slope in degrees
        slope = ee.Terrain.slope(dem)
        # Calculate aspect in degrees
        aspect = ee.Terrain.aspect(dem)
        # print('Number of dem points:')

        print('Number of ndvi points:', ndvi.getInfo())
        print('Number of evi points:', evi.getInfo())
        print('Number of landsat_ndvi points:', landsat_ndvi.getInfo())
        print('Number of dem points:', dem.getInfo())
        print('Number of slope points:', slope.getInfo())
        print('Number of aspect points:', aspect.getInfo())
        # Stack all the features (Sentinel-1, Sentinel-2, Landsat, DEM)
        feature_stack = sentinel1_vh_final.addBands(sentinel1_vv_final) \
                                    .addBands(ndvi) \
                                    .addBands(evi) \
                                    .addBands(landsat_ndvi) \
                                    .addBands(dem) \
                                    .addBands(slope) \
                                    .addBands(aspect)
        
        # print('Size of feature stack :', feature_stack.getInfo())
        
        # Sample the remote sensing data at GEDI footprint locations
        training_data = feature_stack.sampleRegions(
            collection=gedi,
            scale=100                                                                                                                 ,
            geometries=True
        )
        
        print('Number of training_data points:', training_data.size().getInfo())
        if training_data.size().getInfo() == 0:
            return None
        
        # Train a Random Forest model
        classifier = ee.Classifier.smileRandomForest(50).setOutputMode('REGRESSION')
        
        # print('Before trained_model points')
        # Train the model
        trained_model = classifier.train(
            features=training_data,
            classProperty='agbd',
            inputProperties=feature_stack.bandNames()
        )
        
        # Apply the trained model to predict AGB
        agb_prediction = feature_stack.classify(trained_model)        
        
        return agb_prediction.clip(tile_geometry)

agb_list = []
for tile in tile_list:
    if calculate_agb(tile) != None:
        agb_list.append(calculate_agb(tile))
    
# print("Size of agbs")
for agb in agb_list:
    Map.addLayer(agb, {'min': 0, 'max': 300, 'palette': ['5F9EA0', 'grey', 'yellow', '7CFC00','5F8575', '228B22','008000', '355E3B', '4F7942']}, 
              'Predicted AGB')

Map.centerObject(amazon_region, 5);
Map

Number of GEDI points: 18351
Number of filtered GEDI points: 0
Number of GEDI points: 26173
Number of filtered GEDI points: 0
Number of GEDI points: 7276
Number of filtered GEDI points: 0
Number of GEDI points: 38063
Number of filtered GEDI points: 0
Number of GEDI points: 46349
Number of filtered GEDI points: 7214
Number of sentinel1 points: 2082 2082
Number of sentinel2 points: 547
Number of ndvi points: {'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Number of evi points: {'type': 'Image', 'bands': [{'id': 'EVI', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Number of landsat_ndvi points: {'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Number of dem p

EEException: User memory limit exceeded.